## Project Code (Part 2) : Scrape Data with Selenium

In order to collect more data, we need another package. We need the make some javascript click to let the data show up before I can get them. Beautifulsoup cannot do that, thus Katherine suggest that I can use **selenium**. 

In this part of code, I use selenium to inmitate our operations like opening a webpage and make some clicks. The basic idea of selenium is pretty simple, but when using it, there are some problems. Sometimes when the network is not stable, I get some Exceptions. There are also other reason such as when I tried to scrape the data of this house, this house had already been rented, and the information of this house will disapear. Thus I use the exception hadling method in python and selenium package. Especially in selenium package, there is a wait_until function. I can set a delay time, in this delay day period, selenium will try to run my code several times per second, when selenium successfully runs my code, it stop the delay can execute next line of code. 

However, this scraping method also takes lots of time. I open 5 machines at undergrad lab and each machine tried to get data of 500 houses each time. It still took me about 5 hours to scrape all the needed data. For all parts of coding, this part may contain less code, but spent most time.


In [5]:
"""In case TA may need to run our code, I add up the code to install selenium.
    Thus TA can use this code to install selenium.
"""
import pip

def install(package):
    pip.main(['install', 'selenium'])

install('requests') 

  Using cached selenium-3.0.2-py2.py3-none-any.whl


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
from selenium import webdriver
import requests
import pandas as pd
from collections import defaultdict
import time
import numpy as np
import selenium.webdriver.support.expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


In [7]:
'''
In this section of code, we get the data from csv file, filter the house with nan in Bedrooms
Then, we store the data in lists.
'''
df = pd.read_csv('house_renting_dataset_125.csv',index_col=0)

df = df[np.isfinite(df['Bedrooms'])]

House_add = df['Street'].tolist()
House_zipcode = df['Zipcode'].tolist()
House_price = df['Price'].tolist()
House_bed = df['Bedrooms'].tolist()
House_bath = df['Bathrooms'].tolist()
House_type = df['Home_type'].tolist()
House_long = df['Longitude'].tolist()
House_lat = df['Latitude'].tolist()
House_url = df['House_url'].tolist()

In [9]:
base_url = 'https://www.trulia.com/'

In [14]:
path_to_chromedriver = 'chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [11]:
"""I use this list to store the index of house that fail to scrape the data
"""
fail_list_500to1500 = []

In [ ]:
"""This part is the main scraping part using selenium.
"""
dct = defaultdict(list)
for i in range(500,1500):
    Delay_Time = 12
    try:
        browser.get(base_url + df['House_url'][i])
#         some_object = WebDriverWait(browser, 120).until(EC.element_to_be_located((By.CSS_SELECTOR, 'img[alt=\"Some Button\"]')))
        More_detail = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nearbySubtitle"]/a')))
#         browser.find_element_by_xpath('//*[@id="nearbySubtitle"]/a').click()
        More_detail.click()
        time.sleep(2)
        restaurant = WebDriverWait(browser, Delay_Time).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="mapLayerControlList"]/ul/li[2]/div')))
#         browser.find_element_by_xpath('//*[@id="mapLayerControlList"]/ul/li[2]/div').click()
        restaurant.click()
#         time.sleep(2)
#     xpath2 = '//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul'
        entertainments = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul')))

#         entertainments = browser.find_element_by_xpath('//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul')

        en_list = entertainments.find_elements_by_class_name('mbl')
        try:
            crime = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]')))
        except:
            crime = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="listingHomeDetailsContainer"]/div[3]/div[2]/div[2]/div[1]/div[1]')))
        dct['Crime'].append(crime.text)
        dct['High_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[2]').text)
        dct['Middle_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[4]').text)
        dct['Elementary_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[6]').text)
#         crime = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]')))
# #         dct['Crime'].append(browser.find_element_by_xpath('//*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]').text)
#         dct['Crime'].append(crime.text)
        for j in range(5):
            s = en_list[j].find_element_by_tag_name('strong')
            s_split = s.text.split(' ')
            dct[s_split[1]].append(int(s_split[0]))
        school = browser.find_element_by_xpath('//*[@id="mapLayerControlList"]/ul/li[4]/div/div[1]/a/span/strong')
        dct['School'].append(int(school.text.split(' ')[1]))
        dct['Street'].append(House_add[i])
        dct['ZipCode'].append(House_zipcode[i])
        dct['Price'].append(House_price[i])
        dct['Bedrooms'].append(House_bed[i])
        dct['Bathrooms'].append(House_bath[i])
        dct['Home_type'].append(House_type[i])
        dct['Longitude'].append(House_long[i])
        dct['Latitude'].append(House_lat[i])
        dct['House_url'].append(House_url[i])
        
        print('The house ' + str(i) +' has been sucessfully loaded...')
    except:
        fail_list_500to1500.append(i)
        print(str(i) + 'error')
        pass

The house 500 has been sucessfully loaded...
The house 501 has been sucessfully loaded...
The house 502 has been sucessfully loaded...
The house 503 has been sucessfully loaded...
The house 504 has been sucessfully loaded...
The house 505 has been sucessfully loaded...
The house 506 has been sucessfully loaded...
The house 507 has been sucessfully loaded...
The house 508 has been sucessfully loaded...
The house 509 has been sucessfully loaded...
The house 510 has been sucessfully loaded...
The house 511 has been sucessfully loaded...
The house 512 has been sucessfully loaded...
The house 513 has been sucessfully loaded...
The house 514 has been sucessfully loaded...
The house 515 has been sucessfully loaded...
The house 516 has been sucessfully loaded...
The house 517 has been sucessfully loaded...
The house 518 has been sucessfully loaded...
The house 519 has been sucessfully loaded...
The house 520 has been sucessfully loaded...
The house 521 has been sucessfully loaded...
The house 

In [146]:
"""Show the data(demo)
"""
ndf = pd.DataFrame(data=dct)
ndf

,Bathrooms,Bedrooms,Cafes,Crime,Elementary_School,Groceries,High_School,Home_type,House_url,Latitude,Longitude,Middle_School,Nightlife,Price,Restaurants,School,Shopping,Street,ZipCode
0,1.0,1.0,127,Highest,Below Average,31,Below Average,MultiFamily5Plus,/rental-community/9000073953/Avalon-at-Prudent...,42.349214,-71.081227,Below Average,184,"$3,130 – $4,615",518,7,806,"770 Boylston St, Boston, MA",2199.0
1,1.0,1.0,71,Low,Below Average,19,Below Average,MultiFamily2To4,/rental-community/9000037828/CityView-at-Longw...,42.334389,-71.101236,Average,54,"$2,120 – $4,340",252,6,138,"75 Saint Alphonsus St, Boston, MA",2120.0
2,1.0,1.0,177,Lowest,Below Average,26,Average,MultiFamily5Plus,/rental-community/9000056458/The-West-End-Apar...,42.364359,-71.068154,Average,208,"$2,345 – $3,740",709,7,765,"4 Emerson Pl, Boston, MA",2114.0
3,1.0,2.0,102,Low,Below Average,26,Below Average,MultiFamily2To4,/rental-community/9000077477/St-Germain-Apartm...,42.345390,-71.084340,Below Average,159,"$2,165 – $2,600",458,7,735,"67 Saint Germain St, Boston, MA",2115.0
4,1.0,1.0,173,Lowest,Below Average,33,Below Average,Apartment,/rental-community/9000065857/Ink-Block-300-Har...,42.345188,-71.062667,Below Average,200,"$2,382 – $5,961",682,8,899,"300 Harrison Ave, Boston, MA",2118.0
5,1.0,1.0,91,Low,Below Average,18,Below Average,MultiFamily5Plus,/rental-community/9000077479/Landmark-Square-A...,42.343811,-71.098124,Average,84,"$2,625 – $4,700",331,7,374,"75 Peterborough St, Boston, MA",2215.0
6,2.0,1.0,194,Lowest,Below Average,29,Average,MultiFamily5Plus,/rental-community/9000041709/The-Lofts-at-Atla...,42.353753,-71.053012,Below Average,248,"$2,925 – $7,200",766,7,629,"530 Atlantic Ave, Boston, MA",2210.0
7,2.0,3.0,180,High,Below Average,27,Average,MultiFamily2To4,/rental-community/9000075549/One-Canal-Apartme...,42.363757,-71.058997,Average,218,"$2,559 – $27,253",720,7,617,"1 Canal St, Boston, MA",2114.0
8,1.0,1.0,158,Lowest,Below Average,23,Average,MultiFamily5Plus,/rental-community/9000057470/The-Victor-by-Win...,42.366343,-71.060178,Average,177,"$2,600 – $4,490",610,7,560,"110 Beverly St, Boston, MA",2114.0
9,1.0,1.0,80,Lowest,Below Average,17,Below Average,MultiFamily2To4,/rental-community/9000063276/Flats-on-D-411-D-...,42.344100,-71.044641,Average,90,"$2,324 – $5,183",312,7,177,"411 D St, Boston, MA",2210.0


In [1]:
"""test code
"""
# try:
# dct = defaultdict(list)
# tmp = []
# browser.get(base_url + df['House_url'][0])
# More_detail = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nearbySubtitle"]/a')))
# More_detail.click()
# time.sleep(2)
# restaurant = WebDriverWait(browser, Delay_Time).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="mapLayerControlList"]/ul/li[2]/div')))
# restaurant.click()
# entertainments = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul')))
# en_list = entertainments.find_elements_by_class_name('mbl')
# # crime = WebDriverWait(browser, Delay_Time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]')))
# browser.find_element_by_xpath('//*[@id="listingHomeDetailsContainer"]/div[3]/div[2]/div[2]/div[1]/div[1]')
# xpath_crime = '//*[@id="listingHomeDetailsContainer"]/div[3]/div[2]/div[2]/div[1]/div[1]'
# css_crime = '#listingHomeDetailsContainer > div:nth-child(4) > div.col.lastCol > div.box.boxBasic.mtn.backgroundLowlight.clickable.crime_glance_details > div.boxBody.media.pvs.h5 > div.h5.typeWeightNormal.mediaImg.mediaImgExt.typeHighlight'
# dct['Crime'].append(crime.text)
# dct['High_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[2]').text)
# dct['Middle_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[4]').text)
# dct['Elementary_School'].append(browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[6]').text)
# for i in range(5):
#     s = en_list[i].find_element_by_tag_name('strong')
#     s_split = s.text.split(' ')
#     print(s_split)
# school = browser.find_element_by_xpath('//*[@id="mapLayerControlList"]/ul/li[4]/div/div[1]/a/span/strong')
# print(school.text.split(' ')[1])
# dct['Street'].append(House_add[0])
# dct['ZipCode'].append(House_zipcode[0])
# dct['Price'].append(House_price[0])



# dct['Bedrooms'].append(House_bed[0])
# dct['Bathrooms'].append(House_bath[0])
# dct['Home_type'].append(House_type[0])
# dct['Longitude'].append(House_long[0])
# dct['Latitude'].append(House_lat[0])
# dct['House_url'].append(House_url[0])
# except:
#     print('error')

'test code\n'

In [2]:
"""Test Code
"""
# //*[@id="schoolsAtAGlance"]/div[2]/div[4]
# //*[@id="schoolsAtAGlance"]/div[2]/div[6]
# //*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]
# browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[2]').text
# browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[4]').text
# browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[2]/div[6]').text
# browser.find_element_by_xpath('//*[@id="listingHomeDetailsContainer"]/div[4]/div[2]/div[2]/div[1]/div[1]').text

'Test Code\n'

In [3]:
"""Test Code
"""
# browser.get(base_url + df['house_url'][0])
# #     xpath = '//*[@id="nearbySubtitle"]/a'
# browser.find_element_by_xpath('//*[@id="nearbySubtitle"]/a').click()
# time.sleep(2)
# browser.find_element_by_xpath('//*[@id="mapLayerControlList"]/ul/li[2]/div').click()
# time.sleep(2)
# #     xpath2 = '//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul'
# entertainments = browser.find_element_by_xpath('//*[@id="amenitiesLayer"]/div/div/form/div/div/div/ul')
#     en_list = []


In [4]:
"""Test Code
"""
# school = defaultdict(list)
# for i in range(10):
#     print(i)
#     browser.get(base_url + df['house_url'][i])
#     browser.find_element_by_xpath('//*[@id="schoolsAtAGlance"]/div[3]/a').click()
#     time.sleep(2)
#     browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/thead/tr/th[1]/select/option[1]').click()
#     t = browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/tbody/tr[1]/td')
#     school['high'].append(t.text.split('(')[1].split('/')[0])
    
#     time.sleep(2)
#     browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/thead/tr/th[1]/select/option[2]').click()
#     t = browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/tbody/tr[1]/td')
#     school['middle'].append(t.text.split('(')[1].split('/')[0])
    
#     time.sleep(2)
#     browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/thead/tr/th[1]/select/option[3]').click()
#     t = browser.find_element_by_xpath('/html/body/div[6]/div/div/div[1]/div/div[2]/table/tbody/tr[1]/td')
#     school['elementary'].append(t.text.split('(')[1].split('/')[0])